In [1]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 8.8MB/s 


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
def fix_col_names(df):
  return ['IBOV' if col =='^BVSP' else col.rstrip('.SA') for col in df.columns]

In [19]:
tickers = ['VALE3.SA','ITUB4.SA','PETR4.SA','WEGE3.SA','TAEE11.SA']
numero_ativos = len(tickers)

prices = yf.download(tickers, start='2020-01-01', end='2021-05-17')['Adj Close']
prices.columns = fix_col_names(prices)
prices.dropna(inplace=True)

[*********************100%***********************]  5 of 5 completed


In [20]:
retorno = prices.pct_change().dropna()

In [21]:
retorno.head()

,ITUB4,PETR4,TAEE11,VALE3,WEGE3
Date,,,,,
2020-01-03,-0.010518,-0.008143,-0.001297,-0.007362,-0.012770
2020-01-06,-0.014882,0.011823,0.006169,-0.005934,0.002587
2020-01-07,-0.023199,-0.003895,-0.011939,0.007275,0.007741
2020-01-08,-0.016294,-0.006191,-0.010124,0.000185,-0.037838
2020-01-09,-0.019933,-0.003279,-0.012537,-0.013146,-0.014784


In [22]:
retorno.tail()

,ITUB4,PETR4,TAEE11,VALE3,WEGE3
Date,,,,,
2021-05-10,0.014839,0.013126,-0.021288,-0.006583,-0.033214
2021-05-11,-0.002140,0.018219,0.002096,0.035138,-0.000619
2021-05-12,-0.014653,-0.014712,-0.022490,-0.036978,-0.031589
2021-05-13,0.029017,0.008475,0.019262,-0.016094,0.016949
2021-05-14,0.022207,0.052421,0.052756,-0.017157,0.009434


## 1. Carteira com pesos iguais - Equal-Weighted Portfolio

### 1.1 O vetor de pesos

$$ \textbf{w} = \begin{pmatrix} w_1 \\ w_2 \\ \vdots \\ w_n \end{pmatrix}_{(n\times1)} = [w_1, w_2, ... , w_n]^T$$

$$ \sum_{i=1}^n w_i = 1 $$

In [23]:
numero_ativos

5

In [24]:
pesos =  np.ones(numero_ativos) / numero_ativos

In [9]:
pesos

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
### 1.2 A matriz de retornos

Seja um conjunto de dados de $n$ ativos por $t$ dias temos a matriz de retornos dada por:

$$
\textbf{R} = 
\begin{pmatrix}
R_{11} & \ldots & R_{1n}\\
R_{21} & \ldots & R_{2n}\\
\vdots & \ddots &\vdots\\ 
R_{t1} & \ldots & R_{tn}
\end{pmatrix}_{(t \times n)}
$$

In [25]:
# Exibe a dimensão da matriz de retornos
retorno.shape

(335, 5)

In [26]:
retorno.head()

,ITUB4,PETR4,TAEE11,VALE3,WEGE3
Date,,,,,
2020-01-03,-0.010518,-0.008143,-0.001297,-0.007362,-0.012770
2020-01-06,-0.014882,0.011823,0.006169,-0.005934,0.002587
2020-01-07,-0.023199,-0.003895,-0.011939,0.007275,0.007741
2020-01-08,-0.016294,-0.006191,-0.010124,0.000185,-0.037838
2020-01-09,-0.019933,-0.003279,-0.012537,-0.013146,-0.014784


### 1.3 Retorno do Portfólio

#### 1.3.1 Retorno para uma data específica $d$

##### Para um dia $d$ qualquer, o retorno do portfólio nesse dia é dado por:

$$r_d = w_1.R_1 + w_2.R_2+...+w_n.R_n = \sum_{i=1}^{n} w_i.R_i$$

Onde $n$ é o número de ativos do portfólio

In [29]:
# Calculando o Retorno do Portfólio no dia 12/03/2020
r_ativos = retorno.loc['2021-05-14']
ret_pond = np.zeros(numero_ativos)

print('Ativo  | Peso | Retorno | Retorno Ponderado')
print('-------------------------------------------')
for i in range(numero_ativos):
  ret_pond[i] = pesos[i] * r_ativos[i]
  print(f"{r_ativos.index[i]:6} | {pesos[i]:.0%} | {r_ativos[i]:07.2%}  | {ret_pond[i]:07.2%}")

print(f'\nRetorno do Portfólio no dia: {ret_pond.sum():.2%}')

Ativo  | Peso | Retorno | Retorno Ponderado
-------------------------------------------
ITUB4  | 20% | 002.22%  | 000.44%
PETR4  | 20% | 005.24%  | 001.05%
TAEE11 | 20% | 005.28%  | 001.06%
VALE3  | 20% | -01.72%  | -00.34%
WEGE3  | 20% | 000.94%  | 000.19%

Retorno do Portfólio no dia: 2.39%


##### De forma **matricial** podemos escrever, também, o retorno do porfólio no dia $d$ como:

$$

r_d = \textbf{w}^T.\textbf{R}_d =

\begin{pmatrix} w_1 \\ w_2 \\ \vdots \\ w_n \end{pmatrix}
.

\begin{pmatrix}
R_{d1} & \ldots & R_{dn}
\end{pmatrix} 

$$

In [31]:
# Calculando o Retorno do Portfólio no dia 12/03/2020 através do Produto Escalar: método dot()
retorno.loc['2020-05-14'].dot(pesos)

0.0049038397861297595

In [33]:
pesos.T.dot(retorno.loc['2020-05-14'])

0.0049038397861297595

In [37]:
# Calculando o Retorno do Portfólio no dia 12/03/2020 através do Produto Escalar operador @
retorno.loc['2020-05-14'] @ pesos

0.0049038397861297595

In [39]:
pesos.T @ retorno.loc['2020-05-14']

0.0049038397861297595

#### 1.3.2 Retorno do Portfólio em um dado período

$$\textbf{R}_p = \textbf{w}^T\textbf{R} = 
\begin{pmatrix}
R_{11} & \ldots & R_{1n}\\
R_{21} & \ldots & R_{2n}\\
\vdots & \ddots &\vdots\\ 
R_{t1} & \ldots & R_{tn}
\end{pmatrix}_{(t \times n)}
.
\begin{pmatrix} w_1 \\ w_2 \\ \vdots \\ w_n \end{pmatrix}_{(n \times 1)}
=\begin{pmatrix} R_{p_1} \\ R_{p_2} \\ \vdots \\ R_{p_t} \end{pmatrix}_{(t \times 1)}
$$

In [40]:
pesos.T.dot(retorno.T)

array([-8.01814168e-03, -4.73541495e-05, -4.80358542e-03, -1.40523257e-02,
       -1.27356828e-02,  5.15030627e-03,  1.31942953e-02,  2.62416365e-03,
       -3.34495987e-03,  2.33356996e-03,  1.95440816e-02,  8.68764147e-03,
       -1.41934637e-02,  7.13836977e-03,  9.97659769e-03, -2.32163281e-03,
       -2.93203359e-02,  1.45703386e-02, -3.08354822e-03, -3.66747638e-03,
       -2.03131492e-02,  8.32836675e-03,  9.25140482e-03,  6.80040724e-03,
        1.08564459e-02, -1.40129971e-02, -5.06080251e-03,  2.75770866e-02,
        1.90408685e-02, -6.18681151e-03, -1.06629842e-02,  1.10959547e-02,
       -5.60595327e-03,  2.86934429e-02, -1.77978126e-02, -7.75737493e-03,
       -8.59644170e-02, -2.86745056e-03,  4.17115217e-02, -3.40825873e-03,
        2.62196655e-02, -4.10460776e-02, -4.72955929e-02, -1.26575661e-01,
        8.04466468e-02, -7.04261045e-02, -1.30709827e-01,  1.24772447e-01,
       -1.21702408e-01,  3.77557620e-02, -7.84655772e-02,  1.41029205e-02,
        9.09636281e-03, -

In [41]:
pesos @ retorno.T

Date
2020-01-03   -0.008018
2020-01-06   -0.000047
2020-01-07   -0.004804
2020-01-08   -0.014052
2020-01-09   -0.012736
                ...   
2021-05-10   -0.006624
2021-05-11    0.010539
2021-05-12   -0.024084
2021-05-13    0.011522
2021-05-14    0.023932
Length: 335, dtype: float64

In [42]:
retorno.shape, pesos.shape, retorno.dot(pesos).shape

((335, 5), (5,), (335,))

In [43]:
# Erro por causa das dimensões incorretas
#pesos @ retorno

In [44]:
retorno['Carteira'] = retorno.dot(pesos)

In [45]:
retorno.tail()

,ITUB4,PETR4,TAEE11,VALE3,WEGE3,Carteira
Date,,,,,,
2021-05-10,0.014839,0.013126,-0.021288,-0.006583,-0.033214,-0.006624
2021-05-11,-0.002140,0.018219,0.002096,0.035138,-0.000619,0.010539
2021-05-12,-0.014653,-0.014712,-0.022490,-0.036978,-0.031589,-0.024084
2021-05-13,0.029017,0.008475,0.019262,-0.016094,0.016949,0.011522
2021-05-14,0.022207,0.052421,0.052756,-0.017157,0.009434,0.023932


### 1.4 Retorno e Volatilidade anualizados

In [46]:
anualizado = pd.DataFrame()
anualizado['retorno'] = retorno.mean() * 252
anualizado['volatilidade'] = retorno.std() * np.sqrt(252)

In [47]:
anualizado

,retorno,volatilidade
ITUB4,-0.072357,0.452904
PETR4,0.137280,0.659469
TAEE11,0.338657,0.249326
VALE3,0.729565,0.514994
WEGE3,0.611186,0.543726
Carteira,0.348866,0.376150


In [48]:
fig = px.scatter(data_frame=anualizado,
                  y='retorno', 
                  x='volatilidade', 
                  text=anualizado.index,
                  hover_name=anualizado.index,           
                  title='Risco vs Retorno dos ativos e da Carteira')

fig.add_scatter(y=[anualizado['retorno']['Carteira']], 
                x=[anualizado['volatilidade']['Carteira']], 
                marker=dict(color='purple', size=10, symbol='star'),
                showlegend=False)

fig.update_traces(textposition='top center',
                  hovertemplate='<b>retorno: </b> %{y:.1%}'+
                                '<br><b>volatilidade:</b> %{x:.1%}')

fig.layout.xaxis.tickformat = '.0%'
fig.layout.yaxis.tickformat = '.0%'

fig.show()